### Archived Statements - part 3

Target:

 - us-east = 809,993,275
 - us-west = 809,993,275
 - archives = 3,239,979,516+ 2,159,986,344 = 5,399,965,860

 
Process:

 - bulk import via stream consumer of import/statements_archive3
 - use externalScroller to bulk index at the same time

During the last // import we saw that the throughput ended up dropping to 20Kdocs/s, so the gain of having 2 injectors running concurrently does not seem that useful.
Let's use the remaining MongoDB throughput to do the indexing and avoid having to redo a full scroll+reindex.

#### Run the imports

In [ ]:
##!import.sh -o import -t 24 -l import/statements_archive3 -r archives -z 500 -b / -a -w 100000 -bulk -useScroller > logs/import-archives-step3.log